# Coordinates API

In [13]:
import pandas as pd
import numpy as np
import re

# Load data
df_country = pd.read_csv("clean_country.csv")
df_country["Country"] = df_country["Country"].str.title()
df_country


,id,Country,Date_Comment
0,0,Morocco,Check
1,1,Jamaica,Check
2,2,Belize,Check
3,3,Australia,Check
4,4,Australia,Check
...,...,...,...
6965,6965,Australia,Check
6966,6966,Australia,Check
6967,6967,Usa,Check
6968,6968,Panama,Check


In [14]:
# Create a new df with only the countries, unique values

country_counts_coo = df_country['Country'].value_counts().reset_index()
country_counts_coo.columns = ['Country', 'Count']

country_counts_coo

,Country,Count
0,Usa,2550
1,Australia,1485
2,South Africa,598
3,New Zealand,145
4,Bahamas,137
...,...,...
213,Admiralty Islands,1
214,Federated States Of Micronesia,1
215,Red Sea,1
216,British Isles,1


In [15]:
# importing geopy library and Nominatim class
from geopy.geocoders import Nominatim

def capital_coordinates_lat_api(country):
    try:
        coordinates = Nominatim(user_agent="Geopy Library").geocode(country)
        return coordinates.latitude
    except:
        return None

def capital_coordinates_lng_api(country):
    try:
        coordinates = Nominatim(user_agent="Geopy Library").geocode(country)
        return coordinates.longitude
    except:
        return None



country_counts_coo["coo_lat"] = country_counts_coo["Country"].apply(capital_coordinates_lat_api)
country_counts_coo["coo_lng"] = country_counts_coo["Country"].apply(capital_coordinates_lng_api)



In [16]:
country_counts_coo

,Country,Count,coo_lat,coo_lng
0,Usa,2550,39.783730,-100.445882
1,Australia,1485,-24.776109,134.755000
2,South Africa,598,-28.816624,24.991639
3,New Zealand,145,-41.500083,172.834408
4,Bahamas,137,24.773655,-78.000055
...,...,...,...,...
213,Admiralty Islands,1,-2.103264,146.932545
214,Federated States Of Micronesia,1,8.606235,151.832744
215,Red Sea,1,20.296562,38.534315
216,British Isles,1,54.060000,-4.800000


In [19]:
country_counts_coo.to_csv(r"C:\Users\xavie\OneDrive\Documentos\Ironhack\Week 2\Quest\clean_country_w_coordinates.csv", index=False)

In [20]:
# Create a world map with the incidents by coordinates, size by count of instances
import plotly.express as px

# Determine the top countries
top_countries = country_counts_coo.nlargest(5, 'Count')['Country'].tolist()

# Create a new column for colors
country_counts_coo['Color'] = country_counts_coo['Country'].apply(
    lambda x: 'Top 5' if x in top_countries else "Other"  # Change 'blue' to any color you prefer for others
)

fig = px.scatter_geo(country_counts_coo,
                    lat=country_counts_coo.coo_lat,
                    lon=country_counts_coo.coo_lng,
                    size = "Count",
                    size_max=20,
                    hover_name="Country",
                    color="Color",
                    color_discrete_map={'Top 5': '#ed472b', 'Other': '#636efa'},
                    width=1000,
                    height=500)

fig.update_traces(marker=dict(sizemin=4))


fig.show()